<a href="https://colab.research.google.com/github/tilly963963/pytorch/blob/main/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [202]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [203]:
import pandas as pd

In [204]:
df = pd.read_excel(r"/content/drive/Shareddrives/深度學習課程_碩二/上證指數.xlsx")
df

,名稱,代碼,日期,開盤,最高,最低,收盤,漲跌
0,上證指數,980001,2017-01-03,3105.31,3136.46,3105.31,3135.92,1
1,上證指數,980001,2017-01-04,3133.79,3160.10,3130.11,3158.79,1
2,上證指數,980001,2017-01-05,3157.91,3168.50,3154.28,3165.41,1
3,上證指數,980001,2017-01-06,3163.78,3172.03,3153.03,3154.32,0
4,上證指數,980001,2017-01-09,3148.53,3173.14,3147.74,3171.24,1
...,...,...,...,...,...,...,...,...
177,上證指數,980001,2017-09-21,3364.70,3377.88,3356.88,3357.81,0
178,上證指數,980001,2017-09-22,3347.16,3356.45,3334.98,3352.53,0
179,上證指數,980001,2017-09-25,3344.59,3350.96,3334.94,3341.55,0
180,上證指數,980001,2017-09-26,3336.35,3347.16,3332.60,3343.58,1


In [205]:
import torch 
import torch.utils.data as Data
import numpy as np


In [206]:
class MyDataset(Data.Dataset):
  def __init__(self, file_path, seq_length, train=True, transform=None):
    df = pd.read_excel(file_path)
    print(df)
    df_len = len(df)-1
    df_len -= (df_len%seq_length)
    total_size = df_len//seq_length
    train_size = int(total_size*0.7)

    f = df.iloc[:(train_size*seq_length),3:7]
    l = df.iloc[1:(train_size*seq_length)+1,7:]
    data_size=train_size

    if train == False:
      f = df.iloc[(train_size*seq_length):df_len,3:7]
      l = df.iloc[(train_size*seq_length)+1:df_len+1,7:] 
      data_size=total_size-train_size 
    self.features = np.array(f).astype("float32").reshape(data_size,seq_length,4)  #np初始是64 
    self.labels = np.array(l).astype("int").reshape(data_size,seq_length,1)
    self.transform = transform
  def __len__(self):
    return len(self.labels)
  def __getitem__(self, idx):
    # print(idx)
    if torch.is_tensor(idx):
      idx= idx.tolist()
    X = self.features[idx]
    Y = self.labels[idx]
    # print(X)
    # print(Y)
    if self.transform:
      X = self.transform(X)
      Y = self.transform(Y)
    return X,Y
 

In [207]:
def toTensor(x):
  return torch.tensor(x)

In [208]:
file_path=r"/content/drive/Shareddrives/深度學習課程_碩二/上證指數.xlsx"
train_dataset = MyDataset(file_path,seq_length=5,train=True,transform=toTensor)

test_dataset = MyDataset(file_path,seq_length=5,train=False,transform=toTensor)

       名稱      代碼         日期       開盤       最高       最低       收盤  漲跌
0    上證指數  980001 2017-01-03  3105.31  3136.46  3105.31  3135.92   1
1    上證指數  980001 2017-01-04  3133.79  3160.10  3130.11  3158.79   1
2    上證指數  980001 2017-01-05  3157.91  3168.50  3154.28  3165.41   1
3    上證指數  980001 2017-01-06  3163.78  3172.03  3153.03  3154.32   0
4    上證指數  980001 2017-01-09  3148.53  3173.14  3147.74  3171.24   1
..    ...     ...        ...      ...      ...      ...      ...  ..
177  上證指數  980001 2017-09-21  3364.70  3377.88  3356.88  3357.81   0
178  上證指數  980001 2017-09-22  3347.16  3356.45  3334.98  3352.53   0
179  上證指數  980001 2017-09-25  3344.59  3350.96  3334.94  3341.55   0
180  上證指數  980001 2017-09-26  3336.35  3347.16  3332.60  3343.58   1
181  上證指數  980001 2017-09-27  3340.82  3346.58  3340.30  3344.23   1

[182 rows x 8 columns]
       名稱      代碼         日期       開盤       最高       最低       收盤  漲跌
0    上證指數  980001 2017-01-03  3105.31  3136.46  3105.31  3135.92   1
1    上證指數 

In [209]:
featurs, labels = train_dataset[2:9]
print(featurs.shape)
print(labels.shape)


torch.Size([7, 5, 4])
torch.Size([7, 5, 1])


In [210]:
train_loader = Data.DataLoader(dataset=train_dataset,batch_size=5,shuffle=True)
test_loader = Data.DataLoader(dataset=test_dataset,batch_size=5,shuffle=False)


In [211]:
for i,(seqs, labels) in enumerate(train_loader):
  print(seqs.shape)
seqs

torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])


tensor([[[3148.0200, 3150.4600, 3134.6101, 3140.0100],
         [3148.9900, 3157.0300, 3132.6201, 3156.2100],
         [3152.2400, 3186.9800, 3146.6399, 3147.4500],
         [3138.4399, 3158.0500, 3118.0901, 3157.8701],
         [3157.0000, 3187.8899, 3156.9800, 3185.4399]],

        [[3165.6699, 3178.1799, 3148.1799, 3172.1001],
         [3170.2900, 3180.7900, 3158.6299, 3173.1499],
         [3164.2500, 3164.2500, 3111.2100, 3129.5300],
         [3123.8899, 3145.2700, 3117.4500, 3134.5701],
         [3132.9199, 3152.9500, 3131.4199, 3140.8501]],

        [[3086.7100, 3095.4800, 3081.2800, 3090.6299],
         [3087.1699, 3103.9399, 3063.1499, 3075.6799],
         [3069.3899, 3084.2400, 3050.8401, 3061.9500],
         [3047.5701, 3064.8101, 3022.3000, 3064.0801],
         [3055.3401, 3114.6599, 3052.8301, 3107.8301]],

        [[3105.3101, 3136.4600, 3105.3101, 3135.9199],
         [3133.7900, 3160.1001, 3130.1101, 3158.7900],
         [3157.9099, 3168.5000, 3154.2800, 3165.4099],
    

In [212]:
input_size = 4
hidden_size = 32
num_layer = 1
num_class = 1

In [213]:
import torch.nn as nn
from torch.autograd import Variable

In [214]:
class RNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layer,num_class):
    super(RNN,self).__init__()
    self.hidden_size = hidden_size
    self.num_layer = num_layer
    self.normal = nn.BatchNorm1d(5, affine=True)
    self.lstm = nn.LSTM(input_size,hidden_size,num_layer,batch_first=True)
    self.fc = nn.Linear(hidden_size,num_class)
    self.sigmoid = nn.Sigmoid()
  def forward(self, x):
    h0 = Variable(torch.zeros(num_layer,x.size(0),hidden_size))
    c0 = Variable(torch.zeros(num_layer,x.size(0),hidden_size))
    out = self.normal(x)
    out, _ = self.lstm(out,(h0,c0))
    out = self.fc(out)
    out = self.sigmoid(out)
    return out

In [215]:
rnn = RNN(input_size,hidden_size,num_layer,num_class)
rnn

RNN(
  (normal): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(4, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [216]:
output = rnn(seqs)
output

tensor([[[0.4822],
         [0.4862],
         [0.4860],
         [0.4893],
         [0.4955]],

        [[0.4916],
         [0.4975],
         [0.4912],
         [0.4867],
         [0.4826]],

        [[0.4658],
         [0.4597],
         [0.4567],
         [0.4570],
         [0.4554]],

        [[0.4739],
         [0.4800],
         [0.4878],
         [0.4924],
         [0.4927]],

        [[0.4802],
         [0.4822],
         [0.4867],
         [0.4895],
         [0.4881]]], grad_fn=<SigmoidBackward>)

In [217]:
import torch.optim as opt

In [218]:
loss_fn = nn.BCELoss()
optimizer = opt.Adam(rnn.parameters(),lr=0.001)


In [219]:
for epoch in range(800):
  for i,(seqs, labels) in enumerate(train_loader):
    seqs = Variable(seqs)
    labels = Variable(labels)
    optimizer.zero_grad()
    outputs = rnn(seqs)
    loss = loss_fn(outputs,labels.float())#必須是float
    loss.backward()
    optimizer.step()

    if(epoch+1)%100 == 0:
      print("Epoch:%d,batch:%d,Loss:%.4f"%(epoch+1,i+1,loss))

Epoch:100,batch:1,Loss:0.6135
Epoch:100,batch:2,Loss:0.6743
Epoch:100,batch:3,Loss:0.6959
Epoch:100,batch:4,Loss:0.6334
Epoch:100,batch:5,Loss:0.5863
Epoch:200,batch:1,Loss:0.6066
Epoch:200,batch:2,Loss:0.5686
Epoch:200,batch:3,Loss:0.6414
Epoch:200,batch:4,Loss:0.7435
Epoch:200,batch:5,Loss:0.7634
Epoch:300,batch:1,Loss:0.6104
Epoch:300,batch:2,Loss:0.6543
Epoch:300,batch:3,Loss:0.6588
Epoch:300,batch:4,Loss:0.5492
Epoch:300,batch:5,Loss:0.5873
Epoch:400,batch:1,Loss:0.6340
Epoch:400,batch:2,Loss:0.5614
Epoch:400,batch:3,Loss:0.6119
Epoch:400,batch:4,Loss:0.6128
Epoch:400,batch:5,Loss:0.6745
Epoch:500,batch:1,Loss:0.6492
Epoch:500,batch:2,Loss:0.7739
Epoch:500,batch:3,Loss:0.5840
Epoch:500,batch:4,Loss:0.6098
Epoch:500,batch:5,Loss:0.6303
Epoch:600,batch:1,Loss:0.5694
Epoch:600,batch:2,Loss:0.5983
Epoch:600,batch:3,Loss:0.5395
Epoch:600,batch:4,Loss:0.6794
Epoch:600,batch:5,Loss:0.5555
Epoch:700,batch:1,Loss:0.5806
Epoch:700,batch:2,Loss:0.5875
Epoch:700,batch:3,Loss:0.6021
Epoch:700,

In [220]:
correct = 0
total = 0
for seqs, labels in test_dataset:
  for i,(seqs, labels) in enumerate(train_loader):
    seqs = Variable(seqs)
    # labels = Variable(labels)
    outputs = rnn(seqs)
    preducted=outputs.gt(0.5)

    total += (labels.size(0)*labels.size(1))
    correct += (preducted.int() == labels).sum()
print("Acc:%.3f"%(100.*float(correct)//float(total)))

Acc:65.000
